In [1]:
# This file contains code for using mcp servers as tools for agents

In [ ]:
# Importing libraries
import uuid
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.agents import LlmAgent
from google.adk.runners import Runner, InMemoryRunner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool
from dotenv import load_dotenv
import os
print("Imported all required components")

/home/bot/repos/development/google-adk-practice/venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Imported all required components


In [3]:
# Configuration check
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print("START BUILDING!!")
else:
    print("CONFIGURATION ERROR")

START BUILDING!!


In [ ]:
# Retry config
retry_config=types.HttpRetryOptions(
    attempts=5, 
    exp_base=2, 
    intial_delay=2,
    http_status_code=[429, 500, 501, 503, 504]
)


In [ ]:
# MCP integration with Everything Server
mpc_image_server=McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=[
                "-y",
                
                "@modelcontextprotocol/server-everything"
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30
    )
)

print("MCP Tool created")

In [ ]:
# Creating mcp agent that will interact with the mcp_image_server
image_agent=LlmAgent(
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    name="image_agent",
    instruction="User MCP tool to generate images for user queries",
    tools=[mcp_image_server]
)

In [ ]:
# Defining runner
runner = InMemoryRunner(agent=image_agent)

In [ ]:
response = await runner.run_debug(
    "Generate a sample tiny image", verbose=True
)